In [1]:
import pandas as pd
import numpy as np

In [5]:
size = 1000000
user_chunks = pd.read_json('../data/yelp_academic_dataset_user.json', lines=True,
                    dtype = {
                        "review_count": np.int16,
                        "useful": np.int16,
                        "funny": np.int16,
                        "cool": np.int16,
                        "fans": np.int16,
                        "average_stars": np.float32, # calculations are done on float32 anyways for float16
                        "compliment_hot": np.int16,
                        "compliment_more": np.int16,
                        "compliment_profile": np.int16,
                        "compliment_cute": np.int16,
                        "compliment_list": np.int16,
                        "compliment_note": np.int16,
                        "compliment_plain": np.int16,
                        "compliment_cool": np.int16,
                        "compliment_funny": np.int16,
                        "compliment_writer": np.int16,
                        "compliment_photos": np.int16
                    }, chunksize=size)

In [6]:
chunk_list = []
for chunk in user_chunks:
    # only keep tips of current or former elite users.
    chunk = chunk[chunk.user_id.isin(elite_user_ids)]
    chunk_list.append(chunk)

user = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)


KeyboardInterrupt



# Read businesses
Keep only true restaurants

In [21]:
size = 100000
business_chunks = pd.read_json('../data/yelp_academic_dataset_business.json', lines=True,
                    dtype={
                        "business_id": str,
                        "name": str,
                        "address": str,
                        "city": str,
                        "state": str,
                        "postal code": str,
                        "latitude": float,
                        "longitude": float,
                        "stars": float,
                        "review_count": int,
                        "is_open": int
                    }, chunksize=size)

chunk_list = []
for chunk in business_chunks:
    chunk = chunk[chunk['categories'].notnull()]
    
    # only keep restaurants
    chunk = chunk.loc[[('Restaurant' in x) for x in chunk['categories']]]
    
    # chunk = chunk[chunk['city'] == 'Las Vegas']
    
    # only certain columns
    # chunk = chunk[['business_id', 'name','stars', 'is_open','review_count']]
    
    chunk_list.append(chunk)
    
restaurants = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)


In [22]:
restaurants.state.unique()

array(['CO', 'OR', 'BC', 'OH', 'MA', 'FL', 'TX', 'GA', 'WA', 'KS', 'MN',
       'VA', 'WY', 'KY', 'NH', 'ABE'], dtype=object)

In [27]:
restaurants.groupby('city').size().sort_values(ascending=False)

city
Portland        5737
Vancouver       5029
Austin          4965
Atlanta         4180
Orlando         3749
                ... 
Rockport           1
Happy valley       1
Hapevile           1
Hanover            1
portland           1
Length: 459, dtype: int64

In [30]:
restaurants = restaurants[restaurants.city == 'Portland']

In [34]:
restaurants.to_csv('../data/restaurants.csv')

In [23]:
attributes = list(set([attribute for attributes in restaurants.attributes.values  if attributes is not None for attribute in attributes]))

# Read Reviews

In [35]:
restaurant_ids = restaurants.business_id.values

In [36]:
size = 1000000
review_chunks = pd.read_json('../data/yelp_academic_dataset_review.json', lines=True,
                    dtype={
                    "review_id": str,
                    "user_id": str,
                    "business_id": str,
                    "stars": int,
                    "date": str,
                    "text": str,
                    "useful": int,
                    "funny": int,
                    "cool": int,
                }, chunksize=size)

chunk_list = []
for chunk in review_chunks:
    # only keep tips of current or former elite users.
    chunk = chunk[chunk.business_id.isin(restaurant_ids)]
    chunk_list.append(chunk)

reviews = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [37]:
reviews.shape

(752143, 9)

In [38]:
reviews.to_csv('../data/restaurant_reviews.csv', index=False)

In [30]:
reviews.to_pickle('../data/restaurant_reviews.pkl')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\yelp\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lukas\AppData\Local\Temp/ipykernel_6496/2807565378.py", line 1, in <module>
    reviews.to_pickle('../data/restaurant_reviews.pkl')
  File "C:\ProgramData\Anaconda3\envs\yelp\lib\site-packages\pandas\core\generic.py", line 2957, in to_pickle
    to_pickle(
  File "C:\ProgramData\Anaconda3\envs\yelp\lib\site-packages\pandas\io\pickle.py", line 113, in to_pickle
    pickle.dump(
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\yelp\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above 

TypeError: object of type 'NoneType' has no len()

Preprocess text of reviews.
We want to:
* stem words
* remove extra whitespaces
* lower case

In [31]:
# Load the regular expression library
import re
# Remove punctuation
reviews['text_processed'] = \
reviews['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
reviews['text_processed'] = \
reviews['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
reviews['text_processed'].head()

0    apparently prides osteria had a rough summer a...
1    i've stayed at many marriott and renaissance m...
2    the food is always great here the service from...
3    this place used to be a cool chill place now i...
4    the setting is perfectly adequate and the food...
Name: text_processed, dtype: object

In [32]:
reviews.shape

(5575349, 10)

## Wordcloud
Check the word distributions of the texts.

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
long_string = ','.join(list(reviews['text_processed'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()